In [1]:
# %pip install transformers==4.26.1           # Used to know the token size
# %pip install tiktoken==0.3.3
# %pip install openai==0.27.2
%pip install langchain==0.0.134
%pip install python-dotenv==1.0.0
%pip install faiss-cpu==1.7.3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import dotenv_values
import os

api_keys = dotenv_values('keys.txt')
os.environ['OPENAI_API_KEY']=api_keys['OPENAI_API_KEY']

In [3]:
# Load the Groovy HTML documentation and clean it up
import re

# Load a local html file
with open('groovy.html', 'r') as f:
    text = f.read()

# Remove all HTML tags
cleaned_groovy = re.sub(r'<[^>]*>', '', text)    

# Remove blank lines
cleaned_groovy = re.sub(r'^\s*$', '', cleaned_groovy, flags=re.MULTILINE)

# Remove the \n\n that are left over
cleaned_groovy = cleaned_groovy.replace('\n\n', '\n')

# Save the text
with open('groovy.txt', 'w') as f:
    f.write(cleaned_groovy)


In [4]:
# from langchain.llms import OpenAI
# llm = OpenAI()

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 20,
    length_function = len,
)

texts = text_splitter.create_documents([cleaned_groovy])

len(texts)

591

In [7]:
print(texts[0])
print(texts[1])

page_content='Groovy Language Documentation\n.listingblock a.view-result{float:right;font-weight:normal;text-decoration:none;font-size:.9em;line-height:1.4;margin-top:.15em}.exampleblock.result{display:none}\n//[ view example ]\');view_result_link=p.children().last();view_result_link.on(\'click\',$.proxy(toggle_result_block,znode));});}$(\'document\').ready(insert_result_links);\n//]]>\nIntroduction\nGroovy…\u200b\nis an agile and dynamic language for the Java Virtual Machine\nbuilds upon the strengths of Java but has additional power features inspired by languages like Python, Ruby and Smalltalk\nmakes modern programming features available to Java developers with almost-zero learning curve\nprovides the ability to statically type check and statically compile your code for robustness and performance\nsupports Domain-Specific Languages and other compact syntax so your code becomes easy to read and maintain\nmakes writing shell and build scripts easy with its powerful processing primitiv

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [8]:
# Embed and store the texts
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(texts, embeddings)

In [9]:
query = "How do I use groovy?"

search_results = vectorstore.similarity_search(query)
len(search_results)

4

In [10]:
search_results

[Document(page_content='Download a binary distribution of Groovy and unpack it into some folder on your local file system.\nSet your GROOVY_HOME environment variable to the directory where you unpacked the distribution.\nAdd GROOVY_HOME/bin to your PATH environment variable.\nSet your JAVA_HOME environment variable to point to your JDK. On OS X this is /Library/Java/Home, on other unixes its often /usr/java etc. If you’ve already installed tools like Ant or Maven you’ve probably already done this step.\nYou should now have Groovy installed properly. You can test this by typing the following in a command shell:\ngroovysh\nWhich should create an interactive groovy shell where you can type Groovy statements. Or to run the Swing interactive console type:\ngroovyConsole\nTo run a specific Groovy script type:\ngroovy SomeScript\n3.2. Differences with Java\nGroovy tries to be as natural as possible for Java developers. We’ve\ntried to follow the principle of least surprise when designing Groo

In [11]:
from langchain.llms import OpenAI

llm = OpenAI()

In [35]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=search_results, question=query)

' You can use Groovy by integrating it with Java applications using the GroovyServlet. You can use the Eval class to evaluate Groovy code, or the GroovyShell or GroovyScriptEngine to execute Groovy scripts. You can also utilize the type checking extensions to make scripts safer. In addition, you can use Grape to download Jetty and its dependencies, and the TemplateServlet to display template pages.'

In [36]:
docs_and_scores = vectorstore.similarity_search_with_score(query)
docs_and_scores[0]

(Document(page_content='Now put the .groovy files in, say, the root directory (i.e. where you would put your html files). The GroovyServlet takes care of compiling the .groovy files.\n\nSo for example using tomcat you could edit tomcat/conf/server.xml like this:\n\n&lt;Context path="/groovy" docBase="c:/groovy-servlet"/&gt;\n\nThen access it with http://localhost:8080/groovy/hello.groovy\n\n3.20. Integrating Groovy in a Java application\n\n3.20.1. Groovy integration mechanisms\n\nThe Groovy language proposes several ways to integrate itself into applications (Java or even Groovy) at runtime, from\nthe most basic, simple code execution to the most complete, integrating caching and compiler customization.\n\nAll the examples written in this section are using Groovy, but the same integration mechanisms can be used from\nJava.\n\nEval\n\nThe groovy.util.Eval class is the simplest way to execute Groovy dynamically at runtime. This can be done by calling the me\nmethod:\n\nimport groovy.util